In [1]:
import scrapfly_scrapers.linkedin as linkedin

# enable scrapfly cache
linkedin.BASE_CONFIG["cache"] = True
linkedin.BASE_CONFIG["debug"] = True


In [ ]:

print("running Linkedin scrape and saving results to ./results directory")

# profile_data = await linkedin.scrape_profile(
#     urls=[
#         "https://www.linkedin.com/in/williamhgates"
#     ]
# )

from pprint import pprint
# pprint(profile_data)


# with open(output.joinpath("profile.json"), "w", encoding="utf-8") as file:
#     json.dump(profile_data, file, indent=2, ensure_ascii=False)    

# company_data = await linkedin.scrape_company(
#     urls=[
#         "https://linkedin.com/company/microsoft",
#         "https://linkedin.com/company/google",
#         "https://linkedin.com/company/apple"
#     ]
# )
# with open(output.joinpath("company.json"), "w", encoding="utf-8") as file:
#     json.dump(company_data, file, indent=2, ensure_ascii=False)    

# job_search_data = await linkedin.scrape_job_search(
#     # it include other search parameters, refer to the search pages on browser for more details
#     keyword="Python Developer",
#     location="United States",
#     max_pages=3
# )
# with open(output.joinpath("job_search.json"), "w", encoding="utf-8") as file:
#     json.dump(job_search_data, file, indent=2, ensure_ascii=False)    

# job_data = await linkedin.scrape_jobs(
#     urls=[
#         "https://www.linkedin.com/jobs/view/data-center-engineering-operations-engineer-hyd-infinity-dceo-at-amazon-web-services-aws-4017265505",
#         "https://www.linkedin.com/jobs/view/content-strategist-google-cloud-content-strategy-and-experience-at-google-4015776107",
#         "https://www.linkedin.com/jobs/view/sr-content-marketing-manager-brand-protection-brand-protection-at-amazon-4007942181"
#     ]
# )
# with open(output.joinpath("jobs.json"), "w", encoding="utf-8") as file:
#     json.dump(job_data, file, indent=2, ensure_ascii=False)

In [ ]:
from google_search import search, SearchResult

linkedin_pages = list(search("site:www.linkedin.com/company/ intitle:98point6"))
linkedin_pages

In [ ]:
# Took 26 seconds?!?

company_data = await linkedin.scrape_company(
    urls=[
        linkedin_pages[0].link
    ]
)

pprint(company_data)

In [ ]:
job_results = await linkedin.scrape_job_search(
    keyword="software engineer",
    location="United States",
    max_pages=1
)

pprint(job_results)

In [ ]:
    

# def form_urls_params(keyword, location):
    #     """form the job search URL params"""
    #     params = {
    #         "keywords": quote_plus(keyword),
    #         "location": location,
    #     }
    #     return urlencode(params)

    # first_page_url = "https://www.linkedin.com/jobs/search?" + form_urls_params(keyword, location)
    # first_page = await SCRAPFLY.async_scrape(ScrapeConfig(first_page_url, **BASE_CONFIG))

    # selector = response.selector
    # total_results = selector.xpath("//span[contains(@class, 'job-count')]/text()").get()
    # total_results = int(total_results.replace(",", "").replace("+", "")) if total_results else None
    # data = []
    # for element in selector.xpath("//section[contains(@class, 'results-list')]/ul/li"):
    #     data.append({
    #         "title": element.xpath(".//div/a/span/text()").get().strip(),
    #         "company": element.xpath(".//div/div[contains(@class, 'info')]/h4/a/text()").get().strip(),
    #         "address": element.xpath(".//div/div[contains(@class, 'info')]/div/span/text()").get().strip(),
    #         "timeAdded": element.xpath(".//div/div[contains(@class, 'info')]/div/time/@datetime").get(),
    #         "jobUrl": element.xpath(".//div/a/@href").get().split("?")[0],
    #         "companyUrl": element.xpath(".//div/div[contains(@class, 'info')]/h4/a/@href").get().split("?")[0],
    #         "salary": strip_text(element.xpath(".//span[contains(@class, 'salary')]/text()").get())
    #     })
    # return {"data": data, "total_results": total_results}

def build_job_search_url(keyword, location):
    params = {
        "keywords": linkedin.quote_plus(keyword),
        "location": location,
    }
    return f"https://www.linkedin.com/jobs/search/?keywords={linkedin.urlencode(params)}"

url = build_job_search_url("software engineer", "United States")
url

In [ ]:
# manually scraping the job page so that I can iterate quickly on the xpath

from scrapfly import ScrapeConfig, ScrapflyClient
import os
scrape_config = ScrapeConfig(build_job_search_url("software engineer", "United States"), **linkedin.BASE_CONFIG)
scrape_config

client = ScrapflyClient(key=os.environ["SCRAPFLY_KEY"])
response = client.scrape(scrape_config)
response

In [ ]:
selector = response.selector
total_results = selector.xpath("//span[contains(@class, 'job-count')]/text()").get()
total_results = int(total_results.replace(",", "").replace("+", "")) if total_results else None

print(f"Total results: {total_results}")

data = []
for element in selector.xpath("//section[contains(@class, 'results-list')]/ul/li"):
    data.append({
        "title": element.xpath(".//div/a/span/text()").get().strip(),
        "company": element.xpath(".//div/div[contains(@class, 'info')]/h4/a/text()").get().strip(),
        "address": element.xpath(".//div/div[contains(@class, 'info')]/div/span/text()").get().strip(),
        "timeAdded": element.xpath(".//div/div[contains(@class, 'info')]/div/time/@datetime").get(),
        "jobUrl": element.xpath(".//div/a/@href").get().split("?")[0],
        "companyUrl": element.xpath(".//div/div[contains(@class, 'info')]/h4/a/@href").get().split("?")[0],
        "salary": linkedin.strip_text(element.xpath(".//span[contains(@class, 'salary')]/text()").get())
    })
    print(data[-1])


In [ ]:
from pydantic import BaseModel, HttpUrl
from typing import Optional

class JobListing(BaseModel):
    title: str
    company: str
    address: str
    timeAdded: str
    jobUrl: HttpUrl
    companyUrl: HttpUrl
    salary: Optional[str]

for result in data:
    job = JobListing(**result)
    print()